#USC547 Lab Assignment

#Assignment II – CUDA C++ Programming

##Name: Tavishi Kashyap

##group: 2PA1


##Q1. Identify !, %, and %% used in cell in Google Colab.

Answer

1.   ! is used to run Linux shell (terminal) commands inside a notebook
2.   % is used for single line magic commands in Jupyter/Colab like measuring execution time, checking directories etc (Magic commands are special built-in command in Jupyter/Colab that provides extra functionality not available in standard Python, eg file handling, running shell commands inside cell etc)
3.   %% applies magic command to the entire cell

To sum it up- These commands help in running system level operations, timing code and executing/writng non-python code inside notebook



In [9]:
# Example of %% command
#-- Writes cell contet into a file named hello.cu
%%writefile helllo.cu
#include <stdio.h>
__global__ void hello() {
    printf("Hello GPU\n");
}


Writing helllo.cu


In [10]:
#Examples of !


!pwd #--Shows present working directory




/content


In [58]:
!ls #--Lists files in the current directory

answer		  file.cu    mem_separation	ptx_demo.cu  zero_fix
answer.cu	  helllo.cu  mem_separation.cu	sample_data  zero_fix.cu
debug_example.cu  hello.cu   ptx_demo		solution.cu


In [3]:
#Examples for %
%cd /content






/content


In [ ]:
%time x = sum(range(10_000)) #-- Measures execution time for the line


CPU times: user 161 µs, sys: 20 µs, total: 181 µs
Wall time: 183 µs


##Q2. Identify all key nvidia-smi commands with multiple options

Answer


*   nvidia-smi
*   nvidia-smi -L


*   nvidia-smi -q
*   nvidia-smi -h


*   nvidia-smi --help
*   nvidia-smi -l 1








In [ ]:
!nvidia-smi
#This displays overall GPU status and usage information

Wed Jan 28 17:44:48 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvidia-smi -L
#-----to check how many gpus are present ---------------
#Lists all available gpus

GPU 0: Tesla T4 (UUID: GPU-686da4c4-e8fb-a24b-23a1-632fb50a1cf7)


In [ ]:
!nvidia-smi -q
#---Shows: Detailed gpu usage. For example: Temperature, power usage, memory details, clock speed etc




==============NVSMI LOG==============

Timestamp                                 : Wed Jan 28 17:52:17 2026
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla T4
    Product Brand                         : NVIDIA
    Product Architecture                  : Turing
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number                 

In [ ]:
!nvidia-smi --help
# shows all supported options and flags

NVIDIA System Management Interface -- v550.54.15

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- L

In [13]:
!nvidia-smi -h

NVIDIA System Management Interface -- v580.82.07

NVSMI provides monitoring information for Tesla and select Quadro devices.
The data is presented in either a plain text or an XML format, via stdout or a file.
NVSMI also provides several management operations for changing the device state.

Note that the functionality of NVSMI is exposed through the NVML C-based
library. See the NVIDIA developer website for more information about NVML.
Python wrappers to NVML are also available.  The output of NVSMI is
not guaranteed to be backwards compatible; NVML and the bindings are backwards
compatible.

http://developer.nvidia.com/nvidia-management-library-nvml/
http://pypi.python.org/pypi/nvidia-ml-py/
Supported products:
- Full Support
    - All Tesla products, starting with the Kepler architecture
    - All Quadro products, starting with the Kepler architecture
    - All GRID products, starting with the Kepler architecture
    - GeForce Titan products, starting with the Kepler architecture
- L

In [14]:
!nvidia-smi -l 1 #Updates GPU statistics every 1 second, used for real-time monitoring



Sun Feb 15 17:48:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

##Q3.  Debug common CUDA errors (zero output, incorrect indexing, PTX errors)

Answer.

Zero output causes-

Program ends before GPU finishes → missing cudaDeviceSynchronize().

Output computed on GPU but not copied back to host, etc

In [45]:
%%writefile zero_fix.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void helloKernel() {
    printf("Hello from GPU thread %d\n", threadIdx.x);
}

int main() {
    helloKernel<<<1, 4>>>();
    cudaDeviceSynchronize();   // Fix: wait for GPU-- if not there then GPU output may not appear because the program ends before the kernel finishes.

    printf("Hello from CPU\n");
    return 0;
}

Writing zero_fix.cu


In [47]:
!nvcc -arch=sm_75 zero_fix.cu -o zero_fix && ./zero_fix

Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from CPU


Incorrect Indexing Error
Problem

Using only threadIdx.x ignores block number → wrong results for multiple blocks.

In [ ]:
%%writefile index_fix.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void printID() {
    int id = blockIdx.x * blockDim.x + threadIdx.x; //  Wrong indexing-->int id = threadIdx.x;
    printf("Thread id = %d\n", id);
}

int main() {
    printID<<<2, 4>>>();   // 8 total threads
    cudaDeviceSynchronize();
    return 0;
}

 PTX / Architecture Error
Problem

Compiled for wrong GPU architecture.

In [48]:
%%writefile ptx_demo.cu
#include <stdio.h>

__global__ void testKernel() {
    printf("GPU running correctly\n");
}

int main() {
    testKernel<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing ptx_demo.cu


In [52]:
!nvcc -arch=sm_30 ptx_demo.cu -o ptx_demo ./ptx.demo //wrong compile --causes ptx error

nvcc fatal   : Unsupported gpu architecture 'sm_30'


In [51]:
!nvcc -arch=sm_75 ptx_demo.cu -o ptx_demo ./ptx_demo  //correct compile

nvcc fatal   : Don't know what to do with './ptx_demo'


##Q4. Write a CUDA C/C++ program to demonstrate GPU kernel execution and thread indexing.
a. Launch a CUDA kernel using: 1 block and 8 threads

b. Each thread must print: Hello from GPU thread <global_thread_id>

c. Compute the global thread ID using: global_thread_id = blockIdx.x * blockDim.x + threadIdx.x

d. Clearly separate: Host code (CPU) & Device code (GPU kernel)

In [31]:
%%writefile answer.cu
#include <stdio.h>
#include <cuda_runtime.h>

// ---------------Device code (GPU kernel)-------------------
__global__ void helloKernel() {
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_thread_id);
}

// ------------- Host code (CPU)---------------------------
int main() {
    helloKernel<<<1, 8>>>();
    cudaDeviceSynchronize();

    printf("Hello from CPU\n");
    return 0;
}



Overwriting answer.cu


In [32]:
!nvcc -arch=sm_75 answer.cu -o answer && ./answer

Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7
Hello from CPU


##Q5. Write a CUDA program to demonstrate host and device memory separation.
a. Create an integer array of size 5 on the host (CPU).

b. Allocate corresponding memory on the device (GPU) using cudaMalloc().

c. Copy data from host to device using cudaMemcpy().

d. Launch a kernel where GPU threads print values from device memory.

e. Copy the data back from device to host and print it on CPU.

In [55]:
%%writefile mem_separation.cu
#include <stdio.h>
#include <cuda_runtime.h>

// ---------- Device code (GPU) ----------
__global__ void show(int *d) {
    int i = threadIdx.x;              // 1 block only, so threadIdx.x is enough
    printf("GPU: d[%d] = %d\n", i, d[i]);
}

// ---------- Host code (CPU) ----------
int main() {
    int h[5] = {10, 20, 30, 40, 50};
    int *d;

    cudaMalloc(&d, 5 * sizeof(int));
    cudaMemcpy(d, h, 5 * sizeof(int), cudaMemcpyHostToDevice);

    show<<<1, 5>>>(d);
    cudaDeviceSynchronize();

    cudaMemcpy(h, d, 5 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nCPU after copy back:\n");
    for (int i = 0; i < 5; i++) {
        printf("CPU: h[%d] = %d\n", i, h[i]);
    }

    cudaFree(d);
    return 0;
}

Overwriting mem_separation.cu


In [56]:
!nvcc -arch=sm_75 mem_separation.cu -o mem_separation

In [57]:
!./mem_separation

GPU: d[0] = 10
GPU: d[1] = 20
GPU: d[2] = 30
GPU: d[3] = 40
GPU: d[4] = 50

CPU after copy back:
CPU: h[0] = 10
CPU: h[1] = 20
CPU: h[2] = 30
CPU: h[3] = 40
CPU: h[4] = 50


##Q6. Compare CPU times of List/tuple with Numpy arrays.

Answer.

NumPy arrays give much lower CPU time than list/tuple for large numerical computations because NumPy uses vectorized, compiled operations on contiguous memory, whereas list/tuple computations rely on slow Python-level loops.

In [38]:
import time
import numpy as np

N = 5_000_000

lst = list(range(N))
tup = tuple(range(N))
arr = np.arange(N)

# ---- 1) Sum comparison ----
start = time.time()
s1 = sum(lst)
t_list_sum = time.time() - start

start = time.time()
s2 = sum(tup)
t_tuple_sum = time.time() - start

start = time.time()
s3 = np.sum(arr)
t_numpy_sum = time.time() - start

print("SUM timings (seconds)")
print("List :", t_list_sum)
print("Tuple:", t_tuple_sum)
print("NumPy:", t_numpy_sum)

# ---- 2) Element-wise operation: x*2 ----
start = time.time()
lst2 = [x * 2 for x in lst]
t_list_op = time.time() - start

start = time.time()
arr2 = arr * 2
t_numpy_op = time.time() - start

print("\nELEMENT-WISE x*2 timings (seconds)")
print("List  :", t_list_op)
print("NumPy :", t_numpy_op)

SUM timings (seconds)
List : 0.04225659370422363
Tuple: 0.03932380676269531
NumPy: 0.0031175613403320312

ELEMENT-WISE x*2 timings (seconds)
List  : 0.19726777076721191
NumPy : 0.013377189636230469
